In [1]:
# TODO
# -clean up code and make python classes/methods to be importe and called simply here

In [1]:
import os
os.chdir('../')

In [2]:
import csv
import re
import string
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import numpy as np
import pandas as pd

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /home/gui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/gui/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# import numpy as np
# from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import MultinomialNB

# from src.preprocess import pre_process

In [3]:
# list of word types (nouns and adjectives) to leave in the text
defTags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJS', 'JJR']#, 'RB', 'RBS', 'RBR', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# functions to determine the type of a word
def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']

# transform tag forms
def penn_to_wn(tag):
    if is_adjective(tag):
        return nltk.stem.wordnet.wordnet.ADJ
    elif is_noun(tag):
        return nltk.stem.wordnet.wordnet.NOUN
    elif is_adverb(tag):
        return nltk.stem.wordnet.wordnet.ADV
    elif is_verb(tag):
        return nltk.stem.wordnet.wordnet.VERB
    return nltk.stem.wordnet.wordnet.NOUN

In [4]:
def clean(comment_string, lemmatizer):
        clean_tokens = []
        replacement_count = []

        
        token = comment_string
        
        for thing, tag in nltk.pos_tag(token.split()):
            if (thing not in string.punctuation):
                clean_tokens.append(lemmatizer.lemmatize(thing, penn_to_wn(tag)))
        
        clean_tokens = [word for word in clean_tokens if word not in stopwords.words('english')]
            
        token = ' '.join(clean_tokens)
                
        matches = re.findall(r'\\\w*', token)
        matches = [re.sub(r'\\', '', word) for word in list(set(matches))]
        token = re.sub(r'\\\w*', ' ', token)
        
        token = token + ' '.join(matches)
        
        token = re.sub(r'\[', ' ', token)

        token = re.sub(r'\]', ' ', token)

        
        token = re.sub(r'\n', ' ', token)



        token = re.sub(r'\?', ' ', token)

        token = re.sub(r'\"', ' ', token)


        token = re.sub(r'\!', ' ', token)

        token = re.sub(r'\,', ' ', token)

        token = re.sub(r'\.', ' ', token)

        token = re.sub(r'\:', ' ', token)

        token = re.sub(r'\;', ' ', token)

        token = re.sub(r'\)', ' ', token)

        token = re.sub(r'\(', ' ', token)


        token = re.sub(r"\'", ' ', token)

        token = re.sub(r'\+', ' ', token)

        token = re.sub(r"\-", ' ', token)

        token = re.sub(r"\~", ' ', token)

        token = re.sub(r"\*", ' ', token)

        token = re.sub(r"\&", ' ', token)

        token = re.sub(r"\{", ' ', token)

        token = re.sub(r"\}", ' ', token)

        token = re.sub(r"\|", ' ', token)

        token = re.sub(r"\/", ' ', token)

        token = re.sub(r"\#", ' # ', token)

        token = re.sub(' +', ' ', token)

    
    
        token = re.sub(r' 200\d ', ' [year]', token)

        token = re.sub(r' 20\d\d ', ' [year]', token)

        token = re.sub(r' 199\d ', ' [year]', token)


      
        token = re.sub(r' \d+', ' [number] ', token)


        token = token.lower()
        
#         token = re.sub(r'\$( )*\$', ' ', token)
# 
        
#         token = re.sub(r'\$.*\$', ' ', token)
# 

        token = re.sub(r'\_', ' ', token)

            
        return ' '.join(token.split()), replacement_count

In [5]:
def pre_process(file_path, data = 'train', vectorizer = 'tfidf', max_features = None, existing_vectorizer=None):    
    ARXIV = ['astro-ph', 'astro-ph.CO', 'astro-ph.GA', 'astro-ph.SR',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci', 'cs.LG', 'gr-qc',
       'hep-ph', 'hep-th', 'math.AP', 'math.CO', 'physics.optics',
       'quant-ph', 'stat.ML']
    
    lemmatizer = WordNetLemmatizer()
    
    if existing_vectorizer:
        vectorizer = existing_vectorizer
    else:
        if vectorizer == 'tfidf':
            vectorizer = TfidfVectorizer(max_features = max_features)
        elif vectorizer == 'count':
            vectorizer = CountVectorizer(max_features = max_features)
        elif vectorizer == 'binary':
            vectorizer = CountVectorizer(max_features = max_features, binary = True)


    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file)
        colnames = next(csv_reader)

        print('cleaning...')
        if data == 'train':
            raw_data = [[_, clean(comment, lemmatizer)[0], ARXIV.index(cl)] for _, comment, cl in list(csv_reader)]
            X, y = np.array(raw_data)[:, 1], np.array(raw_data)[:, 2]
            
        elif data == 'test':
            raw_data = [[_, clean(comment, lemmatizer)[0]] for _, comment in list(csv_reader)]
            X, y = np.array(raw_data)[:, 1], None

    if data == 'train':

        print('vectorizing...')
        X = vectorizer.fit_transform(X).toarray()
        
        print('done!')
        return X, y, vectorizer
      
    elif data == 'test':
        print('vectorizing...')
        X = vectorizer.transform(X).toarray()

        print('done!')
        return X, None

In [6]:
# X_train, X_val, y_train, y_val, vectorizer = pre_process('dataset/train.csv',
#                                                          data = 'train',
#                                                          vectorizer = 'tfidf',
#                                                         max_features=None)
# X = np.concatenate((X_train,X_val), axis=0)
# y = np.concatenate((y_train,y_val), axis=0)

X, y, vectorizer = pre_process('dataset/train.csv',
                                data = 'train',
                                vectorizer = 'tfidf')


cleaning...
vectorizing...
done!


In [4]:
class BernoulliNB:

    def __init__(self, alpha=1):
        self.alpha = alpha

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        n_classes = self.n_classes

        # calculate P(C_k) for all k
        self.counts = np.zeros(n_classes)
        for i in y:
            self.counts[int(i)] += 1
        self.counts /= len(y)

        # generate n_features x n_classes matrix
        self.params = np.zeros((n_classes, X.shape[1]))
        for idx in range(len(X)):
            self.params[int(y[idx])] += X[idx]
        self.params += self.alpha #1 # Laplace

        # This is the correct thing to do
        self.class_sums = np.zeros(self.n_classes)
        for i in y:
            self.class_sums[int(i)] += 1
        self.class_sums += self.n_classes*self.alpha # Laplace

        # Now our algorithm is happy
        self.params = self.params / self.class_sums[:, np.newaxis]

    def predict(self, X):
        neg_prob = np.log(1 - self.params)
        # Compute  neg_prob · (1 - X).T  as  ∑neg_prob - X · neg_prob
        jll = np.dot(X, (np.log(self.params) - neg_prob).T)
        jll += np.log(self.counts) + neg_prob.sum(axis=1)
        return np.argmax(jll, axis=1)

In [7]:
print(X.shape)
bnb = MultinomialNB(0.1)
bnb.fit(X, y)

(7500, 25092)


MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [8]:
X = []
y = [] 

In [9]:

X, _ = pre_process('dataset/test.csv',
                    data = 'test',
                    vectorizer = 'tfidf',
                    existing_vectorizer = vectorizer)
print(X.shape)
y_pred = bnb.predict(X).astype(int)
print(y_pred)

cleaning...
vectorizing...
done!
(15000, 25092)
[14  3  3 ...  0  7  4]


In [10]:
import pandas as pd

In [11]:
ARXIV = ['astro-ph', 'astro-ph.CO', 'astro-ph.GA', 'astro-ph.SR',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci', 'cs.LG', 'gr-qc',
       'hep-ph', 'hep-th', 'math.AP', 'math.CO', 'physics.optics',
       'quant-ph', 'stat.ML']

df = pd.DataFrame(data = y_pred)

df.columns = ['Category']
df['Id'] = np.arange(len(df))
df = df.apply (lambda row: ARXIV[int(row['Category'])], axis=1)
df.columns = ['Id', 'Category']
df.to_csv('MNB_gridsearch_output.csv')

/home/gui/.local/share/virtualenvs/competition1-arxiv-0Jd2s1wY/lib/python3.5/site-packages/ipykernel_launcher.py:12: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if sys.path[0] == '':
